In [1]:
import os
import sys
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
# 64 classes of optotypes. Should correspond to the folder names under images/testing and images/training
labels = ["+blank", "+circle", "+diamond", "+square", "2", "3", "5", "6", "8", "9", "apple", "bird", "C", "C-0", "C-45",
          "C-90", "C-135", "C-180", "C-225", "C-270", "C-315", "cake", "car", "circle", "cow", "cup", "D",
          "duck", "E", "E-0", "E-90", "E-180", "E-270", "F", "flat-line", "flat-square",
          "frown-line", "frown-square", "H", "hand", "horse", "house", "K", "L", "N", "O", "P",
          "panda", "phone", "R", "S", "smile-line", "smile-square", "square", "star", "T", "train",
          "tree", "V", "x-blank", "x-circle", "x-diamond", "x-square", "Z"]
img_size = 400
channels = 3
training_dir = 'drive/MyDrive/opt_images/training'
testing_dir = 'drive/MyDrive/opt_images/testing'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def create_datasets_test_train():
    """
    Generates two `tf.data.Dataset` from image files in the project.
    NOTE: images must be put under project root in directory images/testing and images/training
    :return: Two `tf.data.Dataset` objects, one for testing and one for training.
    """
    training_set = image_dataset_from_directory(training_dir,
                                                shuffle=True,
                                                batch_size=32,
                                                image_size=(img_size, img_size))
    testing_set = image_dataset_from_directory(testing_dir,
                                               shuffle=True,
                                               batch_size=32,
                                               image_size=(img_size, img_size))
    return training_set, testing_set


In [6]:
def create_datasets(validation_split=0.2):
    """
    Generates three `tf.data.Dataset` from image files in the project.
    @:param validation_split: float between 0 and 1, fraction of data to reserve for validation.
    NOTE: images must be put under project root in directory images/testing and images/training.
    :return: Three `tf.data.Dataset` objects, testing, training, and validation.
    """
    training_set = image_dataset_from_directory(training_dir,
                                                shuffle=True,
                                                batch_size=32,
                                                validation_split=validation_split,
                                                subset="training",
                                                seed=0,
                                                image_size=(img_size, img_size))
    validation_set = image_dataset_from_directory(training_dir,
                                                  shuffle=False,
                                                  batch_size=32,
                                                  validation_split=validation_split,
                                                  subset="validation",
                                                  seed=0,
                                                  image_size=(img_size, img_size))
    testing_set = image_dataset_from_directory(testing_dir,
                                               shuffle=True,
                                               batch_size=32,
                                               image_size=(img_size, img_size))
    return training_set, validation_set, testing_set

In [ ]:
# Just trying to replicate the same thing that I had last time
def create_model():
    vgg = VGG16(include_top=False, weights='imagenet', input_shape=(img_size, img_size, channels))
    vgg.trainable = False
    inputs = tf.keras.Input(shape=(img_size, img_size, channels))
    x = preprocess_input(inputs)
    x = vgg(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(64, activation=tf.keras.activations.softmax)(x)
    model = tf.keras.Model(inputs, outputs)
    model.summary()

    return model

In [7]:
def create_base_model():
    """
    Creates base of VGG16 transfer learning model with imagenet weights and input shape of the same size in config.
    :return: VGG16 model
    """
    vgg = VGG16(include_top=False, weights="imagenet", input_shape=(img_size, img_size, channels))
    vgg.trainable = False
    return vgg

In [ ]:
def runner():
    """
    Essentially the main function. Feel free to change as you see fit.
    """

    # Uncomment if on arcus servers
    # os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    # os.environ["CUDA_VISIBLE_DEVICES"] = "2"

    training_set, validation_set, testing_set = create_datasets()

    # base_model = create_base_model()

    # inputs = tf.keras.Input(shape=(img_size, img_size, channels))

    # x = base_model(inputs, training=False)
    # x = tf.keras.layers.GlobalAveragePooling2D()(x)
    # x = tf.keras.layers.Dropout(0.2)(x)

    # outputs = tf.keras.layers.Dense(1, activation=tf.keras.activations.softmax)(x)

    # model = tf.keras.Model(inputs, outputs)
    # model.summary()

    # model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # model.fit(training_set, epochs=25, verbose=2, validation_data=validation_set,
    #           callbacks=[WandbCallback(data_type="image", labels=labels), TensorBoard(log_dir=wandb.run.dir)])
    # model.evaluate(testing_set, verbose=1)

In [8]:
training_set, validation_set, testing_set = create_datasets()

Found 1500 files belonging to 64 classes.
Using 1200 files for training.
Found 1500 files belonging to 64 classes.
Using 300 files for validation.
Found 3223 files belonging to 64 classes.


In [ ]:
# View a single example entry from a batch
features, label = iter(training_set).next()
print("example features:", features.shape)
print("example label:", label[0])

example features: (32, 400, 400, 3)
example label: tf.Tensor(11, shape=(), dtype=int32)


### Quick test to see how to use given code

In [ ]:
base_model = create_base_model()

inputs = tf.keras.Input(shape=(img_size, img_size, channels))

x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = keras.layers.Flatten()(x)
X = keras.layers.Dense(128, activation='relu')(x)
X = keras.layers.Dropout(0.5)(X)
X = keras.layers.BatchNormalization()(X)

outputs = tf.keras.layers.Dense(64, activation=tf.keras.activations.softmax)(X)

model = tf.keras.Model(inputs, outputs)
model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

The following Variables were used a Lambda layer's call (tf.nn.convolution_26), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 64) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add_26), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1/bias:0' shape=(64,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_27), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv2/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
It is possible that this

In [ ]:
model.fit(training_set, epochs=15, verbose=2, validation_data=validation_set)

Epoch 1/15
38/38 - 25s - loss: 0.9447 - accuracy: 0.6658 - val_loss: 0.3688 - val_accuracy: 0.8767
Epoch 2/15
38/38 - 25s - loss: 0.9465 - accuracy: 0.6608 - val_loss: 0.3475 - val_accuracy: 0.8633
Epoch 3/15
38/38 - 25s - loss: 0.9089 - accuracy: 0.6700 - val_loss: 0.3409 - val_accuracy: 0.8900
Epoch 4/15
38/38 - 25s - loss: 0.9267 - accuracy: 0.6575 - val_loss: 0.3438 - val_accuracy: 0.8600
Epoch 5/15
38/38 - 25s - loss: 0.9676 - accuracy: 0.6658 - val_loss: 0.3418 - val_accuracy: 0.8867
Epoch 6/15
38/38 - 25s - loss: 0.9276 - accuracy: 0.6783 - val_loss: 0.3576 - val_accuracy: 0.8600
Epoch 7/15
38/38 - 25s - loss: 0.9330 - accuracy: 0.6783 - val_loss: 0.3498 - val_accuracy: 0.8667
Epoch 8/15
38/38 - 25s - loss: 0.8873 - accuracy: 0.6883 - val_loss: 0.3444 - val_accuracy: 0.8567
Epoch 9/15
38/38 - 25s - loss: 0.9082 - accuracy: 0.6842 - val_loss: 0.3354 - val_accuracy: 0.8700
Epoch 10/15
38/38 - 25s - loss: 0.9022 - accuracy: 0.6717 - val_loss: 0.3549 - val_accuracy: 0.8667
Epoch 11/

In [ ]:
model.evaluate(testing_set, verbose=1)

101/101 [==============================] - 55s 533ms/step - loss: 2.6762 - accuracy: 0.3010


[2.676180362701416, 0.30096182227134705]

### Trying VGG16 with different epochs, all layers unfrozen

In [9]:
vgg = VGG16(include_top=False, weights="imagenet", input_shape=(img_size, img_size, channels))
vgg.trainable = True
base_model = vgg

inputs = tf.keras.Input(shape=(img_size, img_size, channels))

x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = keras.layers.Flatten()(x)
X = keras.layers.Dense(128, activation='relu')(x)
X = keras.layers.Dropout(0.5)(X)
X = keras.layers.BatchNormalization()(X)

outputs = tf.keras.layers.Dense(64, activation=tf.keras.activations.softmax)(X)

model = tf.keras.Model(inputs, outputs)
model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

58900480/58889256 [==============================] - 0s 0us/step
The following Variables were used a Lambda layer's call (tf.nn.convolution), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 64) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1/bias:0' shape=(64,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_1), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv2/kernel:0' sh

In [10]:
model.fit(training_set, epochs=60, validation_data=validation_set)

Epoch 1/60
38/38 [==============================] - 226s 4s/step - loss: 4.3538 - accuracy: 0.0275 - val_loss: 2.8420 - val_accuracy: 0.3033
Epoch 2/60
38/38 [==============================] - 24s 609ms/step - loss: 3.7540 - accuracy: 0.0967 - val_loss: 2.2993 - val_accuracy: 0.4767
Epoch 3/60
38/38 [==============================] - 23s 600ms/step - loss: 3.3757 - accuracy: 0.1583 - val_loss: 2.0543 - val_accuracy: 0.5667
Epoch 4/60
38/38 [==============================] - 24s 604ms/step - loss: 3.1331 - accuracy: 0.2050 - val_loss: 1.8614 - val_accuracy: 0.6200
Epoch 5/60
38/38 [==============================] - 24s 609ms/step - loss: 2.8923 - accuracy: 0.2483 - val_loss: 1.6806 - val_accuracy: 0.6467
Epoch 6/60
38/38 [==============================] - 24s 604ms/step - loss: 2.7394 - accuracy: 0.2767 - val_loss: 1.5287 - val_accuracy: 0.6933
Epoch 7/60
38/38 [==============================] - 24s 607ms/step - loss: 2.5333 - accuracy: 0.3150 - val_loss: 1.3814 - val_accuracy: 0.7533
E

In [11]:
model.evaluate(testing_set, verbose=1) #60

101/101 [==============================] - 411s 4s/step - loss: 2.6906 - accuracy: 0.3013


[2.6905651092529297, 0.30127209424972534]

In [ ]:
model.evaluate(testing_set, verbose=1) #70

101/101 [==============================] - 51s 498ms/step - loss: 2.8165 - accuracy: 0.3174


[2.816490411758423, 0.31740614771842957]

In [ ]:
model.evaluate(testing_set, verbose=1) #80

101/101 [==============================] - 54s 524ms/step - loss: 2.6981 - accuracy: 0.3162


[2.69810152053833, 0.3161650598049164]

In [ ]:
model.evaluate(testing_set, verbose=1) #90

101/101 [==============================] - 53s 522ms/step - loss: 2.7421 - accuracy: 0.3159


[2.7420878410339355, 0.3158547878265381]

### Trying VGG19 with different epochs, all layers unfrozen



Intially trained model with 70 epochs and kept increasing it by 10

In [ ]:
vgg = VGG19(include_top=False, weights="imagenet", input_shape=(img_size, img_size, channels))
vgg.trainable = True
base_model = vgg

inputs = tf.keras.Input(shape=(img_size, img_size, channels))

x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = keras.layers.Flatten()(x)
X = keras.layers.Dense(128, activation='relu')(x)
X = keras.layers.Dropout(0.5)(X)
X = keras.layers.BatchNormalization()(X)

outputs = tf.keras.layers.Dense(64, activation=tf.keras.activations.softmax)(X)

model = tf.keras.Model(inputs, outputs)
model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

80150528/80134624 [==============================] - 1s 0us/step
The following Variables were used a Lambda layer's call (tf.nn.convolution_13), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 64) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add_13), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1/bias:0' shape=(64,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_14), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv2/kerne

In [ ]:
model.fit(training_set, epochs=10, validation_data=validation_set)


Epoch 1/10
38/38 [==============================] - 27s 696ms/step - loss: 0.6665 - accuracy: 0.7533 - val_loss: 0.2894 - val_accuracy: 0.8833
Epoch 2/10
38/38 [==============================] - 28s 726ms/step - loss: 0.6628 - accuracy: 0.7517 - val_loss: 0.2871 - val_accuracy: 0.8867
Epoch 3/10
38/38 [==============================] - 29s 758ms/step - loss: 0.6699 - accuracy: 0.7583 - val_loss: 0.3026 - val_accuracy: 0.8767
Epoch 4/10
38/38 [==============================] - 28s 737ms/step - loss: 0.7179 - accuracy: 0.7342 - val_loss: 0.3306 - val_accuracy: 0.8633
Epoch 5/10
38/38 [==============================] - 28s 735ms/step - loss: 0.6666 - accuracy: 0.7525 - val_loss: 0.3101 - val_accuracy: 0.8800
Epoch 6/10
38/38 [==============================] - 29s 743ms/step - loss: 0.6689 - accuracy: 0.7533 - val_loss: 0.3048 - val_accuracy: 0.8767
Epoch 7/10
38/38 [==============================] - 29s 741ms/step - loss: 0.7001 - accuracy: 0.7492 - val_loss: 0.2800 - val_accuracy: 0.8800

### Highest accuracy here

In [ ]:
model.evaluate(testing_set, verbose=1) #70

101/101 [==============================] - 62s 611ms/step - loss: 2.7316 - accuracy: 0.3428


[2.7316198348999023, 0.3428482711315155]

In [ ]:
model.evaluate(testing_set, verbose=1) #80

101/101 [==============================] - 60s 587ms/step - loss: 2.7110 - accuracy: 0.3323


[2.7110321521759033, 0.3322991132736206]

In [ ]:
model.evaluate(testing_set, verbose=1) #90

101/101 [==============================] - 60s 589ms/step - loss: 2.7838 - accuracy: 0.3397


[2.783827066421509, 0.33974558115005493]

In [ ]:
model.evaluate(testing_set, verbose=1) #100

101/101 [==============================] - 61s 595ms/step - loss: 2.8927 - accuracy: 0.3301


[2.8927271366119385, 0.33012720942497253]

### Trying VGG16 with all layers frozen

In [19]:
vgg = VGG16(include_top=False, weights="imagenet", input_shape=(img_size, img_size, channels))
vgg.trainable = True
base_model = vgg

# freeze layers
for layer in base_model.layers[:]:
    layer.trainable = False

inputs = tf.keras.Input(shape=(img_size, img_size, channels))

x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = keras.layers.Flatten()(x)
X = keras.layers.Dense(128, activation='relu')(x)
X = keras.layers.Dropout(0.5)(X)
X = keras.layers.BatchNormalization()(X)

outputs = tf.keras.layers.Dense(64, activation=tf.keras.activations.softmax)(X)

model = tf.keras.Model(inputs, outputs)
model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

The following Variables were used a Lambda layer's call (tf.nn.convolution_39), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 64) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add_39), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1/bias:0' shape=(64,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_40), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv2/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
It is possible that this

In [20]:
for layer in base_model.layers:
  print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f1186e972d0> False
<keras.layers.convolutional.Conv2D object at 0x7f1186e979d0> False
<keras.layers.convolutional.Conv2D object at 0x7f1186ee1d50> False
<keras.layers.pooling.MaxPooling2D object at 0x7f11d031a450> False
<keras.layers.convolutional.Conv2D object at 0x7f11d02b99d0> False
<keras.layers.convolutional.Conv2D object at 0x7f11d0314c50> False
<keras.layers.pooling.MaxPooling2D object at 0x7f11d0098ed0> False
<keras.layers.convolutional.Conv2D object at 0x7f11d00a2fd0> False
<keras.layers.convolutional.Conv2D object at 0x7f11d00e2350> False
<keras.layers.convolutional.Conv2D object at 0x7f11d0076750> False
<keras.layers.pooling.MaxPooling2D object at 0x7f11d0098ad0> False
<keras.layers.convolutional.Conv2D object at 0x7f11981b8ed0> False
<keras.layers.convolutional.Conv2D object at 0x7f11d005c5d0> False
<keras.layers.convolutional.Conv2D object at 0x7f11d00643d0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f1186e9c950>

In [21]:
model.fit(training_set, epochs=70, validation_data=validation_set)

Epoch 1/70
38/38 [==============================] - 25s 613ms/step - loss: 4.4464 - accuracy: 0.0308 - val_loss: 2.8212 - val_accuracy: 0.2633
Epoch 2/70
38/38 [==============================] - 24s 621ms/step - loss: 3.8087 - accuracy: 0.0983 - val_loss: 2.4214 - val_accuracy: 0.4267
Epoch 3/70
38/38 [==============================] - 23s 603ms/step - loss: 3.4491 - accuracy: 0.1583 - val_loss: 2.2143 - val_accuracy: 0.4933
Epoch 4/70
38/38 [==============================] - 23s 599ms/step - loss: 3.1583 - accuracy: 0.1983 - val_loss: 1.9939 - val_accuracy: 0.5533
Epoch 5/70
38/38 [==============================] - 24s 609ms/step - loss: 2.9277 - accuracy: 0.2325 - val_loss: 1.8186 - val_accuracy: 0.5867
Epoch 6/70
38/38 [==============================] - 24s 605ms/step - loss: 2.7298 - accuracy: 0.3000 - val_loss: 1.6577 - val_accuracy: 0.6300
Epoch 7/70
38/38 [==============================] - 24s 621ms/step - loss: 2.6236 - accuracy: 0.3000 - val_loss: 1.4804 - val_accuracy: 0.6433

In [22]:
model.evaluate(testing_set, verbose=1) #70

101/101 [==============================] - 50s 488ms/step - loss: 2.7288 - accuracy: 0.3118


[2.7287697792053223, 0.3118212819099426]

### Trying VGG19 with all layers frozen

In [23]:
vgg = VGG19(include_top=False, weights="imagenet", input_shape=(img_size, img_size, channels))
vgg.trainable = True
base_model = vgg

# freeze layers
for layer in base_model.layers[:]:
    layer.trainable = False

inputs = tf.keras.Input(shape=(img_size, img_size, channels))

x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = keras.layers.Flatten()(x)
X = keras.layers.Dense(128, activation='relu')(x)
X = keras.layers.Dropout(0.5)(X)
X = keras.layers.BatchNormalization()(X)

outputs = tf.keras.layers.Dense(64, activation=tf.keras.activations.softmax)(X)

model = tf.keras.Model(inputs, outputs)
model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

80150528/80134624 [==============================] - 0s 0us/step
The following Variables were used a Lambda layer's call (tf.nn.convolution_52), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 64) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add_52), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1/bias:0' shape=(64,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_53), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv2/kerne

In [24]:
model.fit(training_set, epochs=70, validation_data=validation_set)

Epoch 1/70
38/38 [==============================] - 27s 687ms/step - loss: 4.3503 - accuracy: 0.0342 - val_loss: 3.1262 - val_accuracy: 0.2600
Epoch 2/70
38/38 [==============================] - 26s 682ms/step - loss: 3.8167 - accuracy: 0.0825 - val_loss: 2.4633 - val_accuracy: 0.4533
Epoch 3/70
38/38 [==============================] - 27s 689ms/step - loss: 3.4653 - accuracy: 0.1542 - val_loss: 2.1619 - val_accuracy: 0.5333
Epoch 4/70
38/38 [==============================] - 28s 717ms/step - loss: 3.1669 - accuracy: 0.2017 - val_loss: 1.9748 - val_accuracy: 0.5667
Epoch 5/70
38/38 [==============================] - 28s 718ms/step - loss: 2.9550 - accuracy: 0.2275 - val_loss: 1.6929 - val_accuracy: 0.6667
Epoch 6/70
38/38 [==============================] - 27s 706ms/step - loss: 2.7984 - accuracy: 0.2608 - val_loss: 1.4983 - val_accuracy: 0.6800
Epoch 7/70
38/38 [==============================] - 28s 714ms/step - loss: 2.6175 - accuracy: 0.3125 - val_loss: 1.4255 - val_accuracy: 0.6800

In [25]:
model.evaluate(testing_set, verbose=1) #70

101/101 [==============================] - 59s 579ms/step - loss: 2.7424 - accuracy: 0.3124


[2.7424423694610596, 0.3124418258666992]

### Tried ResNet50 for a few different things but training took way longer to get similar results

In [ ]:
from keras.applications.resnet50 import ResNet50

# load model without classifier layers
model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, channels))

# # freeze layers
# for layer in model.layers[:45]:
#     layer.trainable = False

# add new classifier layers
flat1 = keras.layers.Flatten()(model.layers[-1].output)
X = keras.layers.Dense(128, activation='relu')(flat1)
X = keras.layers.Dropout(0.5)(X)
X = keras.layers.BatchNormalization()(X)
output = keras.layers.Dense(64, activation='softmax')(X)

# define new model
model = keras.models.Model(inputs=model.inputs, outputs=output)

# summarize
model.summary()
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(training_set, epochs=40, verbose=2, validation_data=validation_set)

Epoch 1/40
38/38 - 56s - loss: 4.4302 - accuracy: 0.0217 - val_loss: 1477.0565 - val_accuracy: 0.0000e+00
Epoch 2/40
38/38 - 40s - loss: 4.4103 - accuracy: 0.0175 - val_loss: 142.7556 - val_accuracy: 0.0000e+00
Epoch 3/40
38/38 - 40s - loss: 4.4334 - accuracy: 0.0233 - val_loss: 376.6001 - val_accuracy: 0.0000e+00
Epoch 4/40
38/38 - 40s - loss: 4.3460 - accuracy: 0.0200 - val_loss: 13.9761 - val_accuracy: 0.0000e+00
Epoch 5/40
38/38 - 40s - loss: 4.3587 - accuracy: 0.0200 - val_loss: 152.2706 - val_accuracy: 0.0000e+00
Epoch 6/40
38/38 - 40s - loss: 3.9056 - accuracy: 0.0675 - val_loss: 159.3686 - val_accuracy: 0.0667
Epoch 7/40
38/38 - 40s - loss: 3.5100 - accuracy: 0.1300 - val_loss: 16.6092 - val_accuracy: 0.0700
Epoch 8/40
38/38 - 40s - loss: 3.3215 - accuracy: 0.1450 - val_loss: 34.0470 - val_accuracy: 0.0700
Epoch 9/40
38/38 - 40s - loss: 3.1233 - accuracy: 0.1808 - val_loss: 7.3462 - val_accuracy: 0.0767
Epoch 10/40
38/38 - 40s - loss: 2.8063 - accuracy: 0.2458 - val_loss: 2.282

In [ ]:
model.evaluate(testing_set, verbose=1)

101/101 [==============================] - 43s 420ms/step - loss: 3.4276 - accuracy: 0.2650


[3.4275729656219482, 0.2649705111980438]